# Example 05: Rate Limiting and Parallel Epoch Control

This example demonstrates rate limiting and parallel epoch control in `netrun`:

- Using `rate_limit_per_second` to limit how fast epochs execute
- Using `max_parallel_epochs` to limit concurrent epochs per node
- Combining both limits for fine-grained control

In [ ]:
#|default_exp 05_rate_limiting

In [ ]:
#|export
import time
from netrun import (
    Graph, Node, Edge, Port, PortType, PortRef, PortState,
    MaxSalvos, SalvoCondition, SalvoConditionTerm,
    Net, NetState,
)

In [ ]:
#|export
# Create a simple pipeline: Source -> Processor -> Sink
source_node = Node(
    name="Source",
    out_ports={"out": Port()},
    out_salvo_conditions={
        "send": SalvoCondition(
            MaxSalvos.infinite(),
            "out",
            SalvoConditionTerm.port("out", PortState.non_empty())
        )
    }
)

processor_node = Node(
    name="Processor",
    in_ports={"in": Port()},
    out_ports={"out": Port()},
    in_salvo_conditions={
        "receive": SalvoCondition(
            MaxSalvos.finite(1),
            "in",
            SalvoConditionTerm.port("in", PortState.non_empty())
        )
    },
    out_salvo_conditions={
        "send": SalvoCondition(
            MaxSalvos.infinite(),
            "out",
            SalvoConditionTerm.port("out", PortState.non_empty())
        )
    }
)

sink_node = Node(
    name="Sink",
    in_ports={"in": Port()},
    in_salvo_conditions={
        "receive": SalvoCondition(
            MaxSalvos.finite(1),
            "in",
            SalvoConditionTerm.port("in", PortState.non_empty())
        )
    }
)

edges = [
    Edge(
        PortRef("Source", PortType.Output, "out"),
        PortRef("Processor", PortType.Input, "in")
    ),
    Edge(
        PortRef("Processor", PortType.Output, "out"),
        PortRef("Sink", PortType.Input, "in")
    ),
]

graph = Graph([source_node, processor_node, sink_node], edges)

In [ ]:
#|export
def run_rate_limiting_example():
    """Demonstrate rate limiting."""
    print("=" * 60)
    print("Rate Limiting Example")
    print("=" * 60)

    net = Net(
        graph,
        consumed_packet_storage=True,
        on_error="raise",
    )

    # Track execution times
    processor_times = []
    results = []

    def source_exec(ctx, packets):
        """Create 5 packets to be processed."""
        for i in range(5):
            pkt = ctx.create_packet({"id": i})
            ctx.load_output_port("out", pkt)
            ctx.send_output_salvo("send")
        print(f"Source: Created 5 packets")

    def processor_exec(ctx, packets):
        """Process packets with rate limiting."""
        timestamp = time.time()
        processor_times.append(timestamp)

        for port_name, pkts in packets.items():
            for pkt in pkts:
                value = ctx.consume_packet(pkt)
                out_pkt = ctx.create_packet({**value, "processed": True})
                ctx.load_output_port("out", out_pkt)
                ctx.send_output_salvo("send")
                print(f"Processor: Processed packet {value['id']} at t={timestamp:.3f}")

    def sink_exec(ctx, packets):
        """Collect results."""
        for port_name, pkts in packets.items():
            for pkt in pkts:
                results.append(ctx.consume_packet(pkt))

    net.set_node_exec("Source", source_exec)
    net.set_node_exec("Processor", processor_exec)
    net.set_node_exec("Sink", sink_exec)

    # Set rate limit: 5 epochs per second (0.2s between executions)
    net.set_node_config("Processor", rate_limit_per_second=5.0)
    print(f"Rate limit set: 5 epochs/second (200ms interval)")

    net.inject_source_epoch("Source")

    start_time = time.time()
    net.start()
    elapsed = time.time() - start_time

    print(f"\nResults: {len(results)} packets processed")
    print(f"Total time: {elapsed:.3f}s")

    # Calculate intervals between processor executions
    if len(processor_times) > 1:
        intervals = [processor_times[i] - processor_times[i-1]
                     for i in range(1, len(processor_times))]
        avg_interval = sum(intervals) / len(intervals)
        print(f"Average interval between processor executions: {avg_interval*1000:.1f}ms")

    return results

In [ ]:
#|export
def run_max_parallel_epochs_example():
    """Demonstrate max parallel epochs limiting."""
    print("\n" + "=" * 60)
    print("Max Parallel Epochs Example")
    print("=" * 60)

    net = Net(
        graph,
        consumed_packet_storage=True,
        on_error="raise",
    )

    # Track concurrent executions
    concurrent_count = [0]
    max_concurrent = [0]
    results = []

    def source_exec(ctx, packets):
        """Create 10 packets to be processed."""
        for i in range(10):
            pkt = ctx.create_packet({"id": i})
            ctx.load_output_port("out", pkt)
            ctx.send_output_salvo("send")
        print(f"Source: Created 10 packets")

    def processor_exec(ctx, packets):
        """Process packets with parallel execution tracking."""
        concurrent_count[0] += 1
        max_concurrent[0] = max(max_concurrent[0], concurrent_count[0])

        for port_name, pkts in packets.items():
            for pkt in pkts:
                value = ctx.consume_packet(pkt)
                out_pkt = ctx.create_packet({**value, "processed": True})
                ctx.load_output_port("out", out_pkt)
                ctx.send_output_salvo("send")

        concurrent_count[0] -= 1

    def sink_exec(ctx, packets):
        """Collect results."""
        for port_name, pkts in packets.items():
            for pkt in pkts:
                results.append(ctx.consume_packet(pkt))

    net.set_node_exec("Source", source_exec)
    net.set_node_exec("Processor", processor_exec)
    net.set_node_exec("Sink", sink_exec)

    # Set max parallel epochs: only 1 at a time
    net.set_node_config("Processor", max_parallel_epochs=1)
    print(f"Max parallel epochs set: 1")

    net.inject_source_epoch("Source")
    net.start()

    print(f"\nResults: {len(results)} packets processed")
    print(f"Max concurrent processor epochs: {max_concurrent[0]}")
    print(f"Expected max (due to limit): 1")

    return results

In [ ]:
#|export
def run_combined_limits_example():
    """Demonstrate combining rate limiting and max parallel epochs."""
    print("\n" + "=" * 60)
    print("Combined Limits Example")
    print("=" * 60)

    net = Net(
        graph,
        consumed_packet_storage=True,
        on_error="raise",
    )

    processor_times = []
    results = []

    def source_exec(ctx, packets):
        """Create 3 packets."""
        for i in range(3):
            pkt = ctx.create_packet({"id": i})
            ctx.load_output_port("out", pkt)
            ctx.send_output_salvo("send")
        print(f"Source: Created 3 packets")

    def processor_exec(ctx, packets):
        """Process with both limits."""
        processor_times.append(time.time())

        for port_name, pkts in packets.items():
            for pkt in pkts:
                value = ctx.consume_packet(pkt)
                out_pkt = ctx.create_packet({**value, "processed": True})
                ctx.load_output_port("out", out_pkt)
                ctx.send_output_salvo("send")
                print(f"Processor: Processed packet {value['id']}")

    def sink_exec(ctx, packets):
        """Collect results."""
        for port_name, pkts in packets.items():
            for pkt in pkts:
                results.append(ctx.consume_packet(pkt))

    net.set_node_exec("Source", source_exec)
    net.set_node_exec("Processor", processor_exec)
    net.set_node_exec("Sink", sink_exec)

    # Set both limits
    net.set_node_config(
        "Processor",
        max_parallel_epochs=1,
        rate_limit_per_second=10.0  # 100ms between executions
    )
    print(f"Limits set: max_parallel_epochs=1, rate_limit=10/second")

    net.inject_source_epoch("Source")

    start_time = time.time()
    net.start()
    elapsed = time.time() - start_time

    print(f"\nResults: {len(results)} packets processed")
    print(f"Total time: {elapsed:.3f}s")

    return results

In [ ]:
if __name__ == "__main__":
    run_rate_limiting_example()
    run_max_parallel_epochs_example()
    run_combined_limits_example()